In [ ]:
import pandas as pd
from google.cloud import bigquery
from pathlib import Path
import os

cwd = Path(os.getcwd())
PROJECT_ROOT = cwd.parent if cwd.name == "notebooks" else cwd
RAW_DIR = PROJECT_ROOT / "data" / "raw"

def descargar_datos_bigquery_histórico():
    """
    Descarga y concatena datos históricos de ventas desde BigQuery.
    - La primera tabla contiene datos desde 2023 hasta junio de 2025.
    - La segunda tabla contiene datos desde julio de 2025 en adelante.
    El archivo se guarda en data/raw con la fecha del último domingo con datos como nomenclatura.
    """
    from datetime import datetime
    print("Iniciando conexión con BigQuery...")
    client = bigquery.Client()
    print("Conexión establecida.")

    # Descargar datos históricos hasta junio 2025
    query1 = """
    SELECT *
    FROM `fleca-del-port.varios.raw_data_bq_forecasting_20250630`
    """
    df1 = client.query(query1).to_dataframe()

    # Descargar datos desde julio 2025 en adelante
    query2 = """
    SELECT 
        fecha, n_factura, zona_de_venta, producto, familia, cantidad, base_imponible, tipo_IVA, total
    FROM `fleca-del-port.fleca_ventas_dia.t_facturas_dia_extendida_2023`
    WHERE fecha >= '2025-07-01'
    """
    df2 = client.query(query2).to_dataframe()

    # Concatenar ambos DataFrames (unión vertical)
    df = pd.concat([df1, df2], ignore_index=True, sort=False)

    # Calcular el último domingo anterior o igual a la última fecha de datos
    if 'fecha' in df.columns:
        fechas = pd.to_datetime(df['fecha'])
        ultima_fecha = fechas.max()
        ultimo_domingo = ultima_fecha - pd.Timedelta(days=(ultima_fecha.weekday() + 1) % 7)
        fecha_nomenclatura = ultimo_domingo.strftime("%Y%m%d")
        print(f"Rango fechas final: {fechas.min()} - {fechas.max()}")
    else:
        fecha_nomenclatura = datetime.now().strftime("%Y%m%d")

    # Guardar el DataFrame combinado en la carpeta data/raw con nomenclatura de último domingo
    output_path = RAW_DIR / f"raw_data_bq_forecasting_{fecha_nomenclatura}.parquet"
    output_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"Guardando archivo en {output_path} ...")
    df.to_parquet(str(output_path), index=False)
    print("Archivo guardado correctamente.")
    return df

In [ ]:
# Descargar datos históricos. Llamar a la función
raw_all_data = descargar_datos_bigquery_histórico()

In [ ]:
# Verificar los datos descargados
raw_all_data_sorted = raw_all_data.sort_values('fecha')
raw_all_data_sorted.tail(5)